# visdom

- visdom을 익히고 mnist cnn에 적용하기
- visdom은 서버를 켜서 사용해야 한다. 

## visdom 사용법

- text
- image
- images
- Line plot

In [1]:
import torch
import torch.nn as nn

import torchvision
import torchvision.datasets as dsets

In [2]:
import visdom

vis = visdom.Visdom()

Setting up a new session...


In [3]:
vis.text('Hello World!', env="main")

'window_3e2b5dd4399734'

In [ ]:
a = torch.randn(3, 200, 200) # RGB 3 컬러로 200 x 200크기의 사진을 랜덤으로 만든다. 

vis.image(a)

'window_3e2b5de6760822'

In [8]:
vis.images(torch.Tensor(3, 3, 28, 28))

'window_3e2b5e3ad28474'